Only needs to be run once when there is a folder /data_new containing segmaps and random samplings should be created from them. Samplings are saved in /data/samples.

In [31]:
import os
import random
from dataset.preprocessing import pil_to_binary, get_surface_pixels, sample_pixels
from PIL import Image
import matplotlib.pyplot as plt
import random
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

In [32]:
def action_to_border(x, length, height):
        if x <= length:
            return (0, x)  #Top border
        elif x <= length + height:
            return (x - length, length - 1)  # Right border
        elif x <= 2 * length + height:
            return (height - 1, 2 * length + height - x)  # Bottom border
        else:
            return (2 * (length + height) - x, 0)  # Left border

def _shoot_ray(binary_image, border, angle):
    x, y = action_to_border(border, 224, 224)
    
    # Check if initial position is valid
    if (x < 0 or x >= binary_image.shape[1] or 
        y < 0 or y >= binary_image.shape[0]):
        return [], None
        
    # Check if starting on obstacle
    if binary_image[y, x] == 1:
        return [], (x, y)

    dx = np.cos(angle)
    dy = np.sin(angle)
    pixels = [(int(round(x)), int(round(y)))]

    while True:
        x += dx
        y += dy
    
        x_int = int(round(x))
        y_int = int(round(y))
    
        if (x_int < 0 or x_int >= binary_image.shape[1] or 
            y_int < 0 or y_int >= binary_image.shape[0]):
            return pixels, None
    
        elif binary_image[y_int, x_int] == 1:
            return pixels, (x_int, y_int)
        
        else:
            pixels.append((x_int, y_int))

In [ ]:
# Directory paths
input_dir = './data_new'
output_dir = './data_with_rays'
sample_dir = "./data_with_rays/samples"

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Iterate through all files in the input directory
for filename in random.sample(os.listdir(input_dir), 2000):
    if filename.endswith('.png'):
        image_number = filename.split('.')[0]
        #create subfolder for image
        os.makedirs(os.path.join(sample_dir, image_number), exist_ok=True)
        # Load the image
        image_path = os.path.join(input_dir, filename)
        image = Image.open(image_path)
        binary_image = pil_to_binary(image)

        surface_pixels = get_surface_pixels(binary_image)

        output_path = os.path.join(output_dir, filename)
        plt.imsave(output_path, binary_image, cmap='gray')
        
        for i in range(100):
            #random number of rays between 1 and 15
            rnd = random.randint(1,20)
            out_image = np.zeros_like(binary_image)
            num_hits = 0
            j = 0

            while (j < rnd or num_hits == 0) and j < 2 * rnd:
                #randomly choose border pixel of image (size 224x224) and angle
                border = random.randint(0, 2 * 448)
                angle = random.randint(0, 360)

                #shoot ray
                pixels, hit = _shoot_ray(binary_image, border, angle)

                for p in pixels:
                    out_image[p[1], p[0]] = -1
                if hit:
                    out_image[hit[1], hit[0]] = 1
                    num_hits += 1
                j += 1
            
            # Create custom colormap for -1,0,1 values
            colors = [(0,0,0), (0.5,0.5,0.5), (1,1,1)]  # black, gray, white
            n_bins = 3  # 3 distinct values only
            custom_cmap = LinearSegmentedColormap.from_list('custom', colors, N=n_bins)

            #save image to generated subfolder
            sample_path = os.path.join(sample_dir, image_number, f'{i}.png')
            plt.imsave(sample_path, out_image, cmap=custom_cmap, vmin=-1, vmax=1)

print("Sampling completed and saved to", sample_dir)